In [1]:
import pandas as pd
import random
from caboose_nbr.tifuknn import TIFUKNN
from caboose_nbr.evaluate_recommendation import evaluate

In [2]:
train_baskets = pd.read_csv('data/instacart_30k/train_baskets.csv.gz')
test_baskets = pd.read_csv('data/instacart_30k/test_baskets.csv')
valid_baskets = pd.read_csv('data/instacart_30k/valid_baskets.csv')

In [3]:
all_users = train_baskets['user_id'].tolist()
sample_users = random.sample(all_users,1000)

In [4]:
train_baskets = train_baskets[train_baskets['user_id'].isin(sample_users)]
test_baskets = test_baskets[test_baskets['user_id'].isin(sample_users)]
valid_baskets = valid_baskets[valid_baskets['user_id'].isin(sample_users)]

In [5]:
train_baskets.shape

(402890, 10)

In [6]:
train_baskets[['user_id','item_id']].drop_duplicates().shape

(124524, 2)

In [7]:
tifu_sklearn_cosine = TIFUKNN(train_baskets, test_baskets, valid_baskets, mode = 'sklearn', distance_metric = 'cosine')
tifu_sklearn_cosine.train()
tifu_sklearn_cosine_preds = tifu_sklearn_cosine.predict()

number of test users: 964
filtered items: 19958
initial data processing
item count: 8219
compute user reps 964
(964, 8219)
start of knn
knn finished


In [8]:
tifu_caboose = TIFUKNN(train_baskets, test_baskets, valid_baskets,'caboose')
tifu_caboose.train()
tifu_caboose_preds = tifu_caboose.predict()

number of test users: 964
filtered items: 19958
initial data processing
item count: 8219
compute user reps 964
(964, 8219)
start of knn
knn finished


In [9]:
interactions_to_forget = train_baskets[['user_id','item_id']].drop_duplicates().sample(5000)

In [10]:
for user,item in interactions_to_forget.values.tolist():
    if item in tifu_caboose.item_id_mapper and user in tifu_caboose.user_map:
        print(user,item)
        tifu_caboose.user_reps[tifu_caboose.user_map[user]][tifu_caboose.item_id_mapper[item]] = 0
        tifu_caboose.caboose.forget(tifu_caboose.user_map[user],tifu_caboose.item_id_mapper[item])

142544 8277
151345 15040


thread '<unnamed>' panicked at 'called `Option::unwrap()` on a `None` value', /Users/mozhdeh/Documents/caboose_index/src/sparse_topk_index.rs:107:25
note: run with `RUST_BACKTRACE=1` environment variable to display a backtrace


PanicException: called `Option::unwrap()` on a `None` value

In [ ]:
tifu_caboose.forget_interactions(interactions_to_forget.values.tolist())
tifu_caboose_preds_after_forget = tifu_caboose.predict()

In [ ]:
tifu_sklearn_cosine.forget_interactions(interactions_to_forget.values.tolist())
tifu_sklearn_cosine_preds_after_forget = tifu_sklearn_cosine.predict()